# 1. Leer el video

In [1]:
import cv2

In [2]:
cap = cv2.VideoCapture('4_abr_2020_1729.mp4')

# 2. Analizar por milisegundo

In [3]:
import cv2
import torch
from ultralytics import YOLO

model = YOLO('yolo11.pt')  # Asegúrate de que el modelo esté disponible

# Diccionario de nombres de clases
CLASSES = model.names  # Esto es un diccionario con índices de clase como claves

# Función para contar objetos detectados
def count_objects(results):
    # Inicializar el contador de vehículos
    vehicle_count = {name: 0 for name in CLASSES.values()}  
    
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # Obtener la clase detectada
            class_name = CLASSES.get(cls, "unknown")  # Obtener el nombre de la clase
            
            if class_name in vehicle_count:
                vehicle_count[class_name] += 1
    
    return vehicle_count

# Procesar video
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Realizar detección en el frame
        results = model(frame)
        
        # Contar los vehículos
        counts = count_objects(results)
        
        # Imprimir conteo
        print(f"Carros: {counts.get('car', 0)}, Camiones: {counts.get('truck', 0)}, Motos: {counts.get('motorbike', 0)}")
        
        # Mostrar el frame con detecciones
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0])
                class_name = CLASSES.get(cls, "unknown")  # Obtener el nombre
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        cv2.imshow("Detección", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Ejecutar
process_video('4_abr_2020_1729.mp4')



0: 384x640 7 cars, 1 bus, 1 truck, 80.0ms
Speed: 9.3ms preprocess, 80.0ms inference, 8.3ms postprocess per image at shape (1, 3, 384, 640)
Carros: 7, Camiones: 1, Motos: 0

0: 384x640 7 cars, 1 bus, 1 truck, 109.1ms
Speed: 2.9ms preprocess, 109.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Carros: 7, Camiones: 1, Motos: 0

0: 384x640 7 cars, 1 bus, 1 truck, 60.6ms
Speed: 1.7ms preprocess, 60.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)
Carros: 7, Camiones: 1, Motos: 0

0: 384x640 8 cars, 1 bus, 2 trucks, 54.7ms
Speed: 2.0ms preprocess, 54.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Carros: 8, Camiones: 2, Motos: 0

0: 384x640 1 person, 8 cars, 1 motorcycle, 1 bus, 2 trucks, 58.6ms
Speed: 2.2ms preprocess, 58.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Carros: 8, Camiones: 2, Motos: 0

0: 384x640 8 cars, 1 motorcycle, 1 bus, 2 trucks, 49.8ms
Speed: 1.6ms preprocess, 49.8ms inference, 0.9ms 

2025-03-11 14:24:37.689 python[5258:8256999] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-11 14:24:37.689 python[5258:8256999] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Carros: 8, Camiones: 0, Motos: 0

0: 384x640 1 person, 8 cars, 1 motorcycle, 1 bus, 1 truck, 55.5ms
Speed: 1.6ms preprocess, 55.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Carros: 8, Camiones: 1, Motos: 0

0: 384x640 1 person, 7 cars, 1 motorcycle, 1 bus, 53.8ms
Speed: 1.9ms preprocess, 53.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Carros: 7, Camiones: 0, Motos: 0

0: 384x640 1 person, 6 cars, 2 buss, 1 truck, 47.3ms
Speed: 1.5ms preprocess, 47.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Carros: 6, Camiones: 1, Motos: 0

0: 384x640 8 cars, 1 bus, 2 trucks, 56.1ms
Speed: 1.7ms preprocess, 56.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Carros: 8, Camiones: 2, Motos: 0

0: 384x640 11 cars, 1 bus, 2 trucks, 63.0ms
Speed: 1.7ms preprocess, 63.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Carros: 11, Camiones: 2, Motos: 0

0: 384x640 7 cars, 1 bus, 2 trucks, 62.8ms
Spee

KeyboardInterrupt: 

In [7]:
# Define lanes (for simplicity, assume we divide the frame vertically into two parts)
def get_lane(x1, x2, frame_width):
    middle = frame_width // 2  # Assuming the middle of the frame divides the lanes
    if x2 < middle:
        return 'north-south'  # Left side of the frame (north-south lane)
    elif x1 > middle:
        return 'south-north'  # Right side of the frame (south-north lane)
    else:
        return 'unknown'  # In case the vehicle crosses lanes

def count_objects(results, frame_width):
    # Initialize vehicle counters for each lane
    vehicle_count = {'north-south': {'car': 0, 'truck': 0, 'motorbike': 0},
                     'south-north': {'car': 0, 'truck': 0, 'motorbike': 0}}
    
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])  # Get the detected class
            class_name = CLASSES.get(cls, "unknown")  # Get class name
            
            if class_name in ['car', 'truck', 'motorbike']:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                lane = get_lane(x1, x2, frame_width)  # Determine the lane
                if lane != 'unknown':
                    vehicle_count[lane][class_name] += 1
    
    return vehicle_count


Prueba 1 Funcional - Zona derecha e izquierda con conteo pero batallamos con el ID de los estacionados.

In [4]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model = YOLO('yolo11.pt')

# Dictionary to store active trackers
trackers = {}
# Set to keep track of all unique vehicle IDs that have been detected
all_detected_ids = set()
# Set to track vehicles that have crossed into left and right zones
vehicles_left = set()
vehicles_right = set()

# Counter for assigning unique IDs
next_id = 0

# Vehicle class IDs in YOLO (COCO dataset)
# 2: car, 3: motorcycle, 5: bus, 7: truck
VEHICLE_CLASSES = [2, 3, 5, 7]

def process_video(video_path):
    global next_id, all_detected_ids, vehicles_left, vehicles_right
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Detection and tracking settings
    detection_frame_interval = 3  # Run detection every X frames
    frame_count = 0
    
    # Use detection-only approach since OpenCV trackers are problematic
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Define the dividing line for left/right zones
        zone_divider = width // 2
        
        # Run detection on selected frames
        if frame_count % detection_frame_interval == 0:
            results = model(frame)
            
            # Store current detected boxes to update trackers
            current_boxes = []
            
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])
                    
                    # Only process vehicles with good confidence
                    if cls in VEHICLE_CLASSES and conf > 0.5:
                        current_box = (x1, y1, x2, y2, cls)
                        current_boxes.append(current_box)
            
            # Update tracking by matching current detections with existing trackers
            # This is a simple matching by IoU
            matched_trackers = set()
            matched_boxes = set()
            
            # First, try to match current detections with existing trackers
            for tracker_id, tracker_info in list(trackers.items()):
                old_box, cls, missing_count = tracker_info
                ox1, oy1, ox2, oy2 = old_box
                
                # Find best matching box
                best_iou = 0.3  # Minimum IoU threshold
                best_match = None
                
                for i, (x1, y1, x2, y2, det_cls) in enumerate(current_boxes):
                    if i in matched_boxes:
                        continue
                        
                    # Calculate IoU
                    iou = calculate_box_iou((ox1, oy1, ox2, oy2), (x1, y1, x2, y2))
                    
                    if iou > best_iou:
                        best_iou = iou
                        best_match = i
                
                if best_match is not None:
                    # Update tracker with new position
                    x1, y1, x2, y2, det_cls = current_boxes[best_match]
                    trackers[tracker_id] = ((x1, y1, x2, y2), cls, 0)  # Reset missing count
                    matched_trackers.add(tracker_id)
                    matched_boxes.add(best_match)
                else:
                    # Increment missing count for this tracker
                    trackers[tracker_id] = (old_box, cls, missing_count + 1)
                    
                    # Remove trackers that have been missing for too long
                    if missing_count + 1 > 5:  # Remove after missing for 5 detections
                        del trackers[tracker_id]
            
            # Add new trackers for unmatched detections
            for i, (x1, y1, x2, y2, cls) in enumerate(current_boxes):
                if i not in matched_boxes:
                    # Create new tracker
                    trackers[next_id] = ((x1, y1, x2, y2), cls, 0)
                    all_detected_ids.add(next_id)  # Add to the set of all detected IDs
                    next_id += 1
        
        # Count vehicles in each zone and track crossing
        left_count = 0
        right_count = 0
        
        for tracker_id, (bbox, cls, _) in trackers.items():
            x1, y1, x2, y2 = bbox
            
            # Calculate center point to determine zone
            center_x = (x1 + x2) // 2
            
            # Check for crossing from left to right or right to left
            if center_x < zone_divider:
                # Vehicle is in the left zone
                left_count += 1
                if tracker_id not in vehicles_left:
                    vehicles_left.add(tracker_id)  # Vehicle enters the left zone
            else:
                # Vehicle is in the right zone
                right_count += 1
                if tracker_id not in vehicles_right:
                    vehicles_right.add(tracker_id)  # Vehicle enters the right zone
            
            # Draw bounding box and ID
            color = (0, 255, 0) if center_x < zone_divider else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Get class name
            class_names = {2: "Car", 3: "Motorcycle", 5: "Bus", 7: "Truck"}
            class_name = class_names.get(cls, f"Class {cls}")
            
            # Display ID and class
            label = f"ID: {tracker_id} {class_name}"
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Draw zone divider
        cv2.line(frame, (zone_divider, 0), (zone_divider, height), (255, 255, 255), 2)
        
        # Display zone counts
        cv2.putText(frame, f"Left Zone: {len(vehicles_left)}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Right Zone: {len(vehicles_right)}", (zone_divider + 10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Display total unique vehicle count
        total_count = len(vehicles_left) + len(vehicles_right)
        cv2.putText(frame, f"Total Vehicles: {total_count}", (width - 250, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        # Print counts to console
        print(f"Left Zone: {len(vehicles_left)} | Right Zone: {len(vehicles_right)} | Total: {total_count}")
        
        # Show the frame
        cv2.imshow("Vehicle Detection and Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame_count += 1
    
    cap.release()
    cv2.destroyAllWindows()

# Helper function to calculate IoU between two bounding boxes
def calculate_box_iou(box1, box2):
    # Box format: (x1, y1, x2, y2)
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    # Calculate area of each box
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    # Calculate coordinates of intersection
    xi1 = max(x1_1, x1_2)
    yi1 = max(y1_1, y1_2)
    xi2 = min(x2_1, x2_2)
    yi2 = min(y2_1, y2_2)
    
    # Calculate intersection area
    inter_width = max(0, xi2 - xi1)
    inter_height = max(0, yi2 - yi1)
    inter_area = inter_width * inter_height
    
    # Calculate IoU
    union_area = area1 + area2 - inter_area
    iou = inter_area / union_area if union_area > 0 else 0
    
    return iou

# Run video processing
process_video('4_abr_2020_1729.mp4')



0: 384x640 7 cars, 1 bus, 1 truck, 80.0ms
Speed: 9.4ms preprocess, 80.0ms inference, 12.2ms postprocess per image at shape (1, 3, 384, 640)
Left Zone: 3 | Right Zone: 2 | Total: 5
Left Zone: 3 | Right Zone: 2 | Total: 5
Left Zone: 3 | Right Zone: 2 | Total: 5

0: 384x640 8 cars, 1 bus, 2 trucks, 50.9ms
Speed: 3.5ms preprocess, 50.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)
Left Zone: 3 | Right Zone: 2 | Total: 5
Left Zone: 3 | Right Zone: 2 | Total: 5
Left Zone: 3 | Right Zone: 2 | Total: 5

0: 384x640 8 cars, 1 motorcycle, 1 bus, 2 trucks, 54.7ms
Speed: 1.5ms preprocess, 54.7ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Left Zone: 4 | Right Zone: 2 | Total: 6
Left Zone: 4 | Right Zone: 2 | Total: 6
Left Zone: 4 | Right Zone: 2 | Total: 6

0: 384x640 1 person, 8 cars, 1 motorcycle, 1 bus, 1 truck, 63.1ms
Speed: 1.7ms preprocess, 63.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Left Zone: 4 | Right Zone: 2 | Total: 6

Cambie unas cosas, sigue igual.

In [5]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLO model
model = YOLO('yolo11.pt')

# Dictionary to store active trackers
trackers = {}
# Set to keep track of all unique vehicle IDs that have been detected
all_detected_ids = set()
# Set to track vehicles that have crossed into left and right zones
vehicles_left = set()
vehicles_right = set()

# Counter for assigning unique IDs
next_id = 0

# Vehicle class IDs in YOLO (COCO dataset)
# 2: car, 3: motorcycle, 5: bus, 7: truck
VEHICLE_CLASSES = [2, 3, 5, 7]

# Define parameters for tracking
MAX_MISSING_FRAMES = 5  # Max number of frames a vehicle can be missing before being removed

def process_video(video_path):
    global next_id, all_detected_ids, vehicles_left, vehicles_right
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Detection and tracking settings
    detection_frame_interval = 3  # Run detection every X frames
    frame_count = 0
    
    # Use detection-only approach since OpenCV trackers are problematic
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Define the dividing line for left/right zones
        zone_divider = width // 2
        
        # Run detection on selected frames
        if frame_count % detection_frame_interval == 0:
            results = model(frame)
            
            # Store current detected boxes to update trackers
            current_boxes = []
            
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])
                    
                    # Only process vehicles with good confidence
                    if cls in VEHICLE_CLASSES and conf > 0.5:
                        current_box = (x1, y1, x2, y2, cls)
                        current_boxes.append(current_box)
            
            # Update tracking by matching current detections with existing trackers
            matched_trackers = set()
            matched_boxes = set()
            
            # Try to match current detections with existing trackers
            for tracker_id, tracker_info in list(trackers.items()):
                old_box, cls, missing_count = tracker_info
                ox1, oy1, ox2, oy2 = old_box
                
                # Find best matching box using IoU (Intersection over Union)
                best_iou = 0.3  # Minimum IoU threshold
                best_match = None
                
                for i, (x1, y1, x2, y2, det_cls) in enumerate(current_boxes):
                    if i in matched_boxes:
                        continue
                        
                    # Calculate IoU
                    iou = calculate_box_iou((ox1, oy1, ox2, oy2), (x1, y1, x2, y2))
                    
                    if iou > best_iou:
                        best_iou = iou
                        best_match = i
                
                if best_match is not None:
                    # Update tracker with new position
                    x1, y1, x2, y2, det_cls = current_boxes[best_match]
                    trackers[tracker_id] = ((x1, y1, x2, y2), cls, 0)  # Reset missing count
                    matched_trackers.add(tracker_id)
                    matched_boxes.add(best_match)
                else:
                    # Increment missing count for this tracker
                    trackers[tracker_id] = (old_box, cls, missing_count + 1)
                    
                    # Remove trackers that have been missing for too long
                    if missing_count + 1 > MAX_MISSING_FRAMES:  # Remove after missing for too many frames
                        del trackers[tracker_id]
            
            # Add new trackers for unmatched detections
            for i, (x1, y1, x2, y2, cls) in enumerate(current_boxes):
                if i not in matched_boxes:
                    # Create new tracker
                    trackers[next_id] = ((x1, y1, x2, y2), cls, 0)
                    all_detected_ids.add(next_id)  # Add to the set of all detected IDs
                    next_id += 1
        
        # Count vehicles in each zone and track crossing
        left_count = 0
        right_count = 0
        
        for tracker_id, (bbox, cls, _) in trackers.items():
            x1, y1, x2, y2 = bbox
            
            # Calculate center point to determine zone
            center_x = (x1 + x2) // 2
            
            # Check for crossing from left to right or right to left
            if center_x < zone_divider:
                # Vehicle is in the left zone
                left_count += 1
                if tracker_id not in vehicles_left:
                    vehicles_left.add(tracker_id)  # Vehicle enters the left zone
            else:
                # Vehicle is in the right zone
                right_count += 1
                if tracker_id not in vehicles_right:
                    vehicles_right.add(tracker_id)  # Vehicle enters the right zone
            
            # Draw bounding box and ID
            color = (0, 255, 0) if center_x < zone_divider else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Get class name
            class_names = {2: "Car", 3: "Motorcycle", 5: "Bus", 7: "Truck"}
            class_name = class_names.get(cls, f"Class {cls}")
            
            # Display ID and class
            label = f"ID: {tracker_id} {class_name}"
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Draw zone divider
        cv2.line(frame, (zone_divider, 0), (zone_divider, height), (255, 255, 255), 2)
        
        # Display zone counts
        cv2.putText(frame, f"Left Zone: {len(vehicles_left)}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Right Zone: {len(vehicles_right)}", (zone_divider + 10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        # Display total unique vehicle count
        total_count = len(vehicles_left) + len(vehicles_right)
        cv2.putText(frame, f"Total Vehicles: {total_count}", (width - 250, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        # Print counts to console
        print(f"Left Zone: {len(vehicles_left)} | Right Zone: {len(vehicles_right)} | Total: {total_count}")
        
        # Show the frame
        cv2.imshow("Vehicle Detection and Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame_count += 1
    
    cap.release()
    cv2.destroyAllWindows()

# Helper function to calculate IoU between two bounding boxes
def calculate_box_iou(box1, box2):
    # Box format: (x1, y1, x2, y2)
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    # Calculate area of each box
    area1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    # Calculate coordinates of intersection
    xi1 = max(x1_1, x1_2)
    yi1 = max(y1_1, y1_2)
    xi2 = min(x2_1, x2_2)
    yi2 = min(y2_1, y2_2)
    
    # Calculate intersection area
    inter_width = max(0, xi2 - xi1)
    inter_height = max(0, yi2 - yi1)
    inter_area = inter_width * inter_height
    
    # Calculate IoU
    union_area = area1 + area2 - inter_area
    iou = inter_area / union_area if union_area > 0 else 0
    
    return iou

# Run video processing
process_video('4_abr_2020_1729.mp4')



0: 384x640 7 cars, 1 bus, 1 truck, 86.6ms
Speed: 2.8ms preprocess, 86.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Left Zone: 3 | Right Zone: 2 | Total: 5
Left Zone: 3 | Right Zone: 2 | Total: 5
Left Zone: 3 | Right Zone: 2 | Total: 5

0: 384x640 8 cars, 1 bus, 2 trucks, 73.3ms
Speed: 13.5ms preprocess, 73.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Left Zone: 3 | Right Zone: 2 | Total: 5
Left Zone: 3 | Right Zone: 2 | Total: 5
Left Zone: 3 | Right Zone: 2 | Total: 5

0: 384x640 8 cars, 1 motorcycle, 1 bus, 2 trucks, 62.0ms
Speed: 2.1ms preprocess, 62.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Left Zone: 4 | Right Zone: 2 | Total: 6
Left Zone: 4 | Right Zone: 2 | Total: 6
Left Zone: 4 | Right Zone: 2 | Total: 6

0: 384x640 1 person, 8 cars, 1 motorcycle, 1 bus, 1 truck, 68.9ms
Speed: 2.3ms preprocess, 68.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Left Zone: 4 | Right Zone: 2 | Total: 6

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

In [2]:
model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

esto jalo

In [7]:
import cv2
from Tracker3 import ObjectCounter  # Importing ObjectCounter from tracker.py

# Define the mouse callback function
def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:  # Check for mouse movement
        point = [x, y]
        print(f"Mouse moved to: {point}")

# Open the video file
cap = cv2.VideoCapture('4_abr_2020_1729.mp4')

# Define region points for counting
region_points = [(135,208), (676, 203)]

# Initialize the object counter
counter = ObjectCounter(
    region=region_points,  # Pass region points
    model="yolo11s.pt",  # Model for object counting
    classes=[0,2,3,5,7],  # Detect only person class
    show_in=True,  # Display in counts
    show_out=True,  # Display out counts
    line_width=2,  # Adjust line width for display
)

# Create a named window and set the mouse callback
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

count = 0

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break
        # If video ends, reset to the beginning
#        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
#        continue
    count += 1
    if count % 2 != 0:  # Skip odd frames
        continue

    frame = cv2.resize(frame, (1020, 500))

    # Process the frame with the object counter
    frame1 = counter.count(frame)
   
    # Show the frame
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):  # Press 'q' to quit
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

Ultralytics Solutions: ✅ {'region': [(135, 208), (676, 203)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'model': 'yolo11s.pt', 'classes': [0, 2, 3, 5, 7], 'line_width': 2}

0: 320x640 8 cars, 1 motorcycle, 3 buss, 2 trucks, 97.8ms
Speed: 1.7ms preprocess, 97.8ms inference, 0.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 8 cars, 2 buss, 2 trucks, 79.4ms
Speed: 2.1ms preprocess, 79.4ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 bus, 2 trucks, 77.0ms
Speed: 2.3ms preprocess, 77.0ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 bus, 2 trucks, 108.4ms
Speed: 1.7ms preprocess, 108.4ms inference, 1.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 9 cars, 1 bus, 2 trucks, 87.1ms
Speed: 1.2ms preprocess, 87.1ms inference, 0.5ms postprocess per image at s

KeyboardInterrupt: 

In [4]:
import cv2
from Tracker3 import ObjectCounter  # Importing ObjectCounter from tracker.py

# Initialize variables to store the region points
region_points = []

# Define the mouse callback function
def RGB(event, x, y, flags, param):
    global region_points
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse button click
        # Store the coordinates of the clicked point
        region_points.append((x, y))
        print(f"Clicked at: {x}, {y}")
        
        # If two points have been clicked, we can set the region
        if len(region_points) == 2:
            print(f"Region set: {region_points[0]} to {region_points[1]}")
            # Initialize the object counter with the selected region
            counter.region = region_points
            # Allow the video analysis to start now
            global analysis_started
            analysis_started = True

# Open the video file
cap = cv2.VideoCapture('4_abr_2020_1729.mp4')

# Initialize the object counter with placeholder region
counter = ObjectCounter(
    region=[(0, 0), (0, 0)],  # Placeholder region, will be updated with clicks
    model="yolo11s.pt",  # Model for object counting
    classes=[0, 2, 3, 5, 7],  # Detect only person class
    show_in=True,  # Display in counts
    show_out=True,  # Display out counts
    line_width=2,  # Adjust line width for display
)

# Create a named window and set the mouse callback
cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

count = 0
analysis_started = False  # Flag to control when the analysis starts

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break

    # Skip video analysis until both points are clicked
    if not analysis_started:
        # Show the instructions or just keep showing the video without processing
        cv2.putText(frame, "Click to set region points (top-left and bottom-right).", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow("RGB", frame)
        cv2.waitKey(1)
        continue  # Wait until both points are clicked

    count += 1
    if count % 2 != 0:  # Skip odd frames
        continue

    frame = cv2.resize(frame, (1020, 500))

    # Process the frame with the object counter
    frame1 = counter.count(frame)
   
    # Show the frame
    cv2.imshow("RGB", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):  # Press 'q' to quit
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


Ultralytics Solutions: ✅ {'region': [(0, 0), (0, 0)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'records': 5, 'model': 'yolo11s.pt', 'classes': [0, 2, 3, 5, 7], 'line_width': 2}
Clicked at: 217, 794
Clicked at: 1546, 672
Region set: (217, 794) to (1546, 672)

0: 320x640 1 person, 12 cars, 101.5ms
Speed: 5.7ms preprocess, 101.5ms inference, 12.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 11 cars, 1 bus, 80.0ms
Speed: 1.3ms preprocess, 80.0ms inference, 0.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 13 cars, 75.0ms
Speed: 1.4ms preprocess, 75.0ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 13 cars, 62.0ms
Speed: 1.7ms preprocess, 62.0ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 13 cars, 68.2ms
Speed: 1.3ms preprocess, 68.2ms inferenc

: 